In [28]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import tensorflow as tf


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

DATADIR = "D:\\PRODIGYINFOTECH\\dogs-vs-cats\\train\\train"  
CATEGORIES = ["cat", "dog"]
IMG_SIZE = 64
MAX_IMAGES_PER_CLASS = 1000  

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


image_paths = []
labels = []
count_cat = count_dog = 0

for file in tqdm(os.listdir(DATADIR), desc="Collecting file paths"):
    if "cat" in file and count_cat < MAX_IMAGES_PER_CLASS:
        image_paths.append(os.path.join(DATADIR, file))
        labels.append(0)
        count_cat += 1
    elif "dog" in file and count_dog < MAX_IMAGES_PER_CLASS:
        image_paths.append(os.path.join(DATADIR, file))
        labels.append(1)
        count_dog += 1
    if count_cat >= MAX_IMAGES_PER_CLASS and count_dog >= MAX_IMAGES_PER_CLASS:
        break

X = []
for path in tqdm(image_paths, desc="Loading and preprocessing images"):
    img = load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
    x = img_to_array(img)
    X.append(x)

X = np.array(X)
X = preprocess_input(X)
y = np.array(labels)

features = base_model.predict(X, batch_size=32, verbose=1)
features = features.reshape(features.shape[0], -1)  # Flatten features

X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)


model = SVC(kernel='rbf', C=1.0)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

print("\n Accuracy:", accuracy_score(y_test, y_pred))
print(" Classification Report:\n", classification_report(y_test, y_pred, target_names=CATEGORIES))

Loading and preprocessing images: 100%|███████████████████████████████████████████| 2000/2000 [00:06<00:00, 291.85it/s]


63/63 ━━━━━━━━━━━━━━━━━━━━ 43s 680ms/step

 Accuracy: 0.82
 Classification Report:
               precision    recall  f1-score   support

         cat       0.82      0.82      0.82       199
         dog       0.82      0.82      0.82       201

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

